In [2]:
%load_ext autoreload
%autoreload 2
from indices import MainDataset
from neighbors import Neighborhoods
import functions
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from catboost import Pool
from metric import metrics_stat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
train_dataset = MainDataset("dataset/train.csv")
test_dataset = MainDataset("dataset/test.csv", need_index = False)

/Users/ivan.bragin/projects/raif/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
neighborhoods = Neighborhoods(train_dataset.index)
neighborhoods1 = Neighborhoods(train_dataset.index1)

In [5]:
# for i, o in enumerate(train_dataset.all_objects[:10000]):
#     if i % 1000 == 0:
#         print(i)
#     neighbor = neighborhoods.get_haversine_closest(o, 7)
#     for f_name, f in functions.items():
#         prices = [f(o, x[0], x[1]) for x in neighbor]
#         o.neighbors_scores[f_name] = np.mean(prices)

In [6]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(train_dataset.all_objects):
#         o.neighbors_scores = {}
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min
    

253000
254000
255000
256000
253000
254000
255000
256000


In [7]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for i, o in enumerate(test_dataset.all_objects):
        if o.row["price_type"] != 1:
            continue
        if i % 1000 == 0:
            print(i)
        neighbor = neighborhoods_item.get_haversine_closest(o, 7)
        for f_name, f in functions.functions.items():
            prices = [f(o, x[0], x[1]) for x in neighbor]
            prices = [x for x in prices if x > 0]
            if len(prices) > 0:
                price_median = np.median(prices)
                price_max = np.max(prices)
                price_min = np.min(prices)

            else:
                price_median = 0
                price_max = 0
                price_min = 0
            o.neighbors_scores[prefix + f_name+"_median"] = price_median
            o.neighbors_scores[prefix + f_name+"_max"] = price_max
            o.neighbors_scores[prefix + f_name+"_min"] = price_min

0
1000
2000
0
1000
2000


In [8]:
raw_dense_features = [
#     'osm_historic_points_in_0.005',
#     'osm_culture_points_in_0.0075',
        'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
        'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
        'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01',
                      'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500',
       'total_square', "lat", "lng"
       ]

raw_cat_features = ['city', 'osm_city_nearest_name', 'region', 'realty_type', 'street', 'floor']

bad_f = set(["F_square_False_min", "F_square_False_median"])
def getX(o):
    x = []
    for f_name, score in o.neighbors_scores.items():
        if f_name not in bad_f and "P1_" not in f_name:
            x.append(score)
    for n in raw_dense_features:
        x.append(o.row[n])
    for n in raw_cat_features:
        x.append(o.row[n])
    return pd.Series(x)


In [9]:
columns = []
for p in [""
#           "P1_"
         ]:
    for t in ["_median", "_min", "_max"]:
        columns.extend([p + f_name + t for f_name, f in functions.functions.items() ])
columns = [c for c in columns if c not in bad_f]
    # columns = []
columns.extend(raw_dense_features)
columns.extend(raw_cat_features)

In [10]:
# X0 = [getX(o) for o in train_dataset.all_objects[:1] if o.row["price_type"] == 0]
# X0 = pd.concat(X0, axis=1).T
# X0.columns=columns
# Y0 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 0]
# X0["floor"] = X0["floor"].astype("str")

In [11]:
X1 = [getX(o) for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = pd.concat(X1, axis=1).T
X1.columns=columns
Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]
X1 = X1.fillna(-999)


In [12]:
Xtest1 = [getX(o) for o in test_dataset.all_objects]
Xtest1 = pd.concat(Xtest1, axis=1).T
Xtest1.columns=columns
Xtest1 = Xtest1.fillna(-999)

In [13]:
# X0 = np.stack([o.getX() for o in train_dataset.all_objects if o.row["price_type"] == 0])
# X1 = np.stack([o.getX() for o in train_dataset.all_objects if o.row["price_type"] == 1])
# Y1 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 1]

In [14]:
# np.save("x0", X0)
# np.save("y0", Y0)
# np.save("x1", X1)
# np.save("y1", Y1)

In [15]:
# X0 = X0.fillna(-999)
X_train, X_val, Y_train, Y_val = train_test_split(X1, Y1, test_size=0.1, random_state=42)

In [16]:

train_data = Pool(data=X_train,
                  cat_features=raw_cat_features,
                  label=Y_train)

val_data = Pool(data=X_val,
                cat_features=raw_cat_features,
                  label=Y_val)

In [17]:
cat_model = CatBoostRegressor(
    learning_rate=0.03,
    iterations=7000
)
cat_model.fit(train_data,eval_set=val_data, use_best_model=False)



0:	learn: 89130.3352965	test: 83591.6983637	best: 83591.6983637 (0)	total: 56ms	remaining: 6m 31s
1:	learn: 88304.4271130	test: 82436.9308489	best: 82436.9308489 (1)	total: 67.3ms	remaining: 3m 55s
2:	learn: 87532.8020843	test: 81641.7094066	best: 81641.7094066 (2)	total: 77ms	remaining: 2m 59s
3:	learn: 86824.2187932	test: 80946.8475988	best: 80946.8475988 (3)	total: 86.6ms	remaining: 2m 31s
4:	learn: 86088.3744899	test: 79999.2461382	best: 79999.2461382 (4)	total: 96.8ms	remaining: 2m 15s
5:	learn: 85384.9634113	test: 79158.9155489	best: 79158.9155489 (5)	total: 107ms	remaining: 2m 4s
6:	learn: 84760.3965806	test: 78547.0517691	best: 78547.0517691 (6)	total: 116ms	remaining: 1m 55s
7:	learn: 84066.2557794	test: 77907.3653390	best: 77907.3653390 (7)	total: 125ms	remaining: 1m 49s
8:	learn: 83401.0486269	test: 77220.4306910	best: 77220.4306910 (8)	total: 135ms	remaining: 1m 44s
9:	learn: 82786.5443657	test: 76440.7598555	best: 76440.7598555 (9)	total: 145ms	remaining: 1m 41s
10:	learn:

88:	learn: 62767.3663369	test: 52623.3628140	best: 52623.3628140 (88)	total: 1.07s	remaining: 1m 22s
89:	learn: 62700.1136093	test: 52616.8638662	best: 52616.8638662 (89)	total: 1.08s	remaining: 1m 22s
90:	learn: 62625.0364311	test: 52597.2839857	best: 52597.2839857 (90)	total: 1.09s	remaining: 1m 22s
91:	learn: 62451.5805026	test: 52531.6388915	best: 52531.6388915 (91)	total: 1.1s	remaining: 1m 22s
92:	learn: 62339.2426150	test: 52333.0709312	best: 52333.0709312 (92)	total: 1.11s	remaining: 1m 22s
93:	learn: 62250.7303427	test: 52040.3217552	best: 52040.3217552 (93)	total: 1.13s	remaining: 1m 22s
94:	learn: 62112.1789607	test: 51979.8876889	best: 51979.8876889 (94)	total: 1.14s	remaining: 1m 22s
95:	learn: 62038.2084723	test: 51974.4960567	best: 51974.4960567 (95)	total: 1.15s	remaining: 1m 22s
96:	learn: 61957.7665957	test: 51962.4353183	best: 51962.4353183 (96)	total: 1.17s	remaining: 1m 22s
97:	learn: 61849.1585637	test: 51918.6288273	best: 51918.6288273 (97)	total: 1.18s	remaining

184:	learn: 54028.2094122	test: 47875.7358622	best: 47875.7358622 (184)	total: 2.07s	remaining: 1m 16s
185:	learn: 53914.4935707	test: 47836.3403295	best: 47836.3403295 (185)	total: 2.09s	remaining: 1m 16s
186:	learn: 53539.5071509	test: 47816.1888259	best: 47816.1888259 (186)	total: 2.1s	remaining: 1m 16s
187:	learn: 53450.2980281	test: 47812.2164151	best: 47812.2164151 (187)	total: 2.11s	remaining: 1m 16s
188:	learn: 53422.3202601	test: 47823.1444025	best: 47812.2164151 (187)	total: 2.12s	remaining: 1m 16s
189:	learn: 53373.4981309	test: 47804.9335621	best: 47804.9335621 (189)	total: 2.13s	remaining: 1m 16s
190:	learn: 53357.3407679	test: 47806.2929086	best: 47804.9335621 (189)	total: 2.14s	remaining: 1m 16s
191:	learn: 53231.7240152	test: 47785.9031344	best: 47785.9031344 (191)	total: 2.15s	remaining: 1m 16s
192:	learn: 53177.5116736	test: 47778.7782368	best: 47778.7782368 (192)	total: 2.16s	remaining: 1m 16s
193:	learn: 53119.4079357	test: 47656.2078081	best: 47656.2078081 (193)	to

268:	learn: 47692.6744942	test: 45445.7813027	best: 45436.8234378 (266)	total: 2.89s	remaining: 1m 12s
269:	learn: 47644.1652569	test: 45443.9283837	best: 45436.8234378 (266)	total: 2.9s	remaining: 1m 12s
270:	learn: 47614.4133786	test: 45439.4528546	best: 45436.8234378 (266)	total: 2.91s	remaining: 1m 12s
271:	learn: 47591.6736719	test: 45365.5161931	best: 45365.5161931 (271)	total: 2.92s	remaining: 1m 12s
272:	learn: 47546.8899289	test: 45348.7789505	best: 45348.7789505 (272)	total: 2.93s	remaining: 1m 12s
273:	learn: 47493.6991064	test: 45279.3677340	best: 45279.3677340 (273)	total: 2.94s	remaining: 1m 12s
274:	learn: 47473.1003055	test: 45189.8081461	best: 45189.8081461 (274)	total: 2.94s	remaining: 1m 12s
275:	learn: 47391.6978127	test: 45186.4841461	best: 45186.4841461 (275)	total: 2.96s	remaining: 1m 12s
276:	learn: 47245.1818218	test: 45208.7282723	best: 45186.4841461 (275)	total: 2.97s	remaining: 1m 12s
277:	learn: 46998.5931510	test: 45208.7938904	best: 45186.4841461 (275)	to

351:	learn: 42864.8202133	test: 43689.0859668	best: 43689.0859668 (351)	total: 3.7s	remaining: 1m 9s
352:	learn: 42851.8208025	test: 43686.8227280	best: 43686.8227280 (352)	total: 3.71s	remaining: 1m 9s
353:	learn: 42751.6560587	test: 43682.4145619	best: 43682.4145619 (353)	total: 3.72s	remaining: 1m 9s
354:	learn: 42713.1834297	test: 43671.0014992	best: 43671.0014992 (354)	total: 3.73s	remaining: 1m 9s
355:	learn: 42678.0614486	test: 43613.2800516	best: 43613.2800516 (355)	total: 3.74s	remaining: 1m 9s
356:	learn: 42549.6743277	test: 43606.7310859	best: 43606.7310859 (356)	total: 3.75s	remaining: 1m 9s
357:	learn: 42454.2095010	test: 43606.4038211	best: 43606.4038211 (357)	total: 3.76s	remaining: 1m 9s
358:	learn: 42439.6396060	test: 43607.5002037	best: 43606.4038211 (357)	total: 3.77s	remaining: 1m 9s
359:	learn: 42389.7038152	test: 43541.5030178	best: 43541.5030178 (359)	total: 3.78s	remaining: 1m 9s
360:	learn: 42323.3057971	test: 43551.6944368	best: 43541.5030178 (359)	total: 3.79

...

6624:	learn: 4714.8240739	test: 39462.7254232	best: 39452.7824320 (6227)	total: 1m 4s	remaining: 3.63s
6625:	learn: 4714.6646628	test: 39462.8235628	best: 39452.7824320 (6227)	total: 1m 4s	remaining: 3.63s
6626:	learn: 4714.2736093	test: 39463.1759843	best: 39452.7824320 (6227)	total: 1m 4s	remaining: 3.62s
6627:	learn: 4713.2308683	test: 39462.8334177	best: 39452.7824320 (6227)	total: 1m 4s	remaining: 3.61s
6628:	learn: 4712.9916642	test: 39462.8543074	best: 39452.7824320 (6227)	total: 1m 4s	remaining: 3.6s
6629:	learn: 4712.8976732	test: 39462.7951351	best: 39452.7824320 (6227)	total: 1m 4s	remaining: 3.59s
6630:	learn: 4712.7117228	test: 39462.0631122	best: 39452.7824320 (6227)	total: 1m 4s	remaining: 3.58s
6631:	learn: 4711.1010625	test: 39462.1193920	best: 39452.7824320 (6227)	total: 1m 4s	remaining: 3.57s
6632:	learn: 4710.8146933	test: 39461.1742914	best: 39452.7824320 (6227)	total: 1m 4s	remaining: 3.56s
6633:	learn: 4710.0581978	test: 39460.8451010	best: 39452.7824320 (6227)	t

6712:	learn: 4646.0776401	test: 39464.5658856	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 2.78s
6713:	learn: 4645.2914186	test: 39465.1956401	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 2.77s
6714:	learn: 4644.7633965	test: 39465.2299113	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 2.76s
6715:	learn: 4644.1163088	test: 39464.5611137	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 2.75s
6716:	learn: 4643.3241994	test: 39464.5320054	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 2.74s
6717:	learn: 4642.8486347	test: 39464.7605507	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 2.73s
6718:	learn: 4642.0685950	test: 39464.5993390	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 2.72s
6719:	learn: 4641.1950289	test: 39464.6213631	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 2.71s
6720:	learn: 4640.7811135	test: 39464.3262191	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 2.7s
6721:	learn: 4640.5604040	test: 39463.7022930	best: 39452.7824320 (6227)	t

6799:	learn: 4573.1757334	test: 39467.1522139	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 1.94s
6800:	learn: 4572.1577013	test: 39467.9011320	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 1.93s
6801:	learn: 4572.1071365	test: 39467.9877514	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 1.92s
6802:	learn: 4570.8620941	test: 39468.2122624	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 1.91s
6803:	learn: 4569.8233851	test: 39468.8846635	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 1.9s
6804:	learn: 4569.4581381	test: 39469.0719753	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 1.89s
6805:	learn: 4568.7622801	test: 39468.3755771	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 1.88s
6806:	learn: 4567.5031495	test: 39468.6704962	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 1.87s
6807:	learn: 4566.8578677	test: 39469.0831969	best: 39452.7824320 (6227)	total: 1m 5s	remaining: 1.86s
6808:	learn: 4564.6992788	test: 39468.4877074	best: 39452.7824320 (6227)	t

6887:	learn: 4497.7279214	test: 39456.5774135	best: 39452.7824320 (6227)	total: 1m 6s	remaining: 1.08s
6888:	learn: 4495.6769592	test: 39456.6170820	best: 39452.7824320 (6227)	total: 1m 6s	remaining: 1.07s
6889:	learn: 4494.7382403	test: 39455.8707265	best: 39452.7824320 (6227)	total: 1m 6s	remaining: 1.06s
6890:	learn: 4493.7009350	test: 39455.6380813	best: 39452.7824320 (6227)	total: 1m 6s	remaining: 1.05s
6891:	learn: 4492.8113433	test: 39456.4715898	best: 39452.7824320 (6227)	total: 1m 6s	remaining: 1.04s
6892:	learn: 4491.1208104	test: 39455.5352845	best: 39452.7824320 (6227)	total: 1m 6s	remaining: 1.03s
6893:	learn: 4490.4166885	test: 39455.4224845	best: 39452.7824320 (6227)	total: 1m 6s	remaining: 1.03s
6894:	learn: 4488.7618308	test: 39455.5216156	best: 39452.7824320 (6227)	total: 1m 6s	remaining: 1.02s
6895:	learn: 4488.4039639	test: 39455.3637219	best: 39452.7824320 (6227)	total: 1m 6s	remaining: 1.01s
6896:	learn: 4487.5859277	test: 39455.0184430	best: 39452.7824320 (6227)	

6975:	learn: 4428.3264614	test: 39452.2714325	best: 39450.9005030 (6956)	total: 1m 7s	remaining: 232ms
6976:	learn: 4428.2377325	test: 39452.3900585	best: 39450.9005030 (6956)	total: 1m 7s	remaining: 223ms
6977:	learn: 4427.1711856	test: 39452.2117868	best: 39450.9005030 (6956)	total: 1m 7s	remaining: 213ms
6978:	learn: 4426.2469591	test: 39452.0288673	best: 39450.9005030 (6956)	total: 1m 7s	remaining: 203ms
6979:	learn: 4426.2330611	test: 39451.9599941	best: 39450.9005030 (6956)	total: 1m 7s	remaining: 193ms
6980:	learn: 4425.3521726	test: 39451.9866643	best: 39450.9005030 (6956)	total: 1m 7s	remaining: 184ms
6981:	learn: 4424.7530044	test: 39452.8285696	best: 39450.9005030 (6956)	total: 1m 7s	remaining: 174ms
6982:	learn: 4423.1128766	test: 39452.8241031	best: 39450.9005030 (6956)	total: 1m 7s	remaining: 164ms
6983:	learn: 4421.5216967	test: 39451.9903472	best: 39450.9005030 (6956)	total: 1m 7s	remaining: 155ms
6984:	learn: 4421.2554065	test: 39451.7407679	best: 39450.9005030 (6956)	

In [18]:
pd.set_option("display.max_rows", None) 
cat_model.get_feature_importance(data=val_data,prettified=True)


,Feature Id,Importances
0,total_square,8.874865
1,reform_mean_floor_count_500,4.871857
2,osm_crossing_points_in_0.005,4.607044
3,F_square_True_max,3.137083
4,street,2.717787
5,region,2.692701
6,osm_catering_points_in_0.0075,2.647057
7,osm_catering_points_in_0.005,2.309998
8,osm_catering_points_in_0.001,2.285539
9,realty_type,2.275232


In [104]:
price0_preds = cat_model.predict(X1)
metrics_stat(np.array(price0_preds) , np.array(Y1))

{'mape': 0.07674616213711428,
 'mdape': 0.05781454409175535,
 'rmse': 9831.006113200872,
 'r2': 0.9865341223767325,
 'raif_metric': 0.04891707267317312}

In [28]:
price0_preds = cat_model.predict(X_val, ntree_end=7000)
metrics_stat(np.array(Y_val), np.array(price0_preds)-000)

{'mape': 0.2567656300724026,
 'mdape': 0.1783382697672723,
 'rmse': 39718.01122245567,
 'r2': 0.7790814849711565,
 'raif_metric': 1.3236600532090184}

In [ ]:
{'mape': 0.32152263584077434,
 'mdape': 0.23299677152276008,
 'rmse': 87343.51408296668,
 'r2': -1.4305272953932735,
 'raif_metric': 1.9368490908547882}

In [ ]:
{'mape': 0.3938137450820849,
 'mdape': 0.27994944633239843,
 'rmse': 93738.73635776664,
 'r2': -3.7693751005637317,
 'raif_metric': 2.492385311141158}

{'mape': 0.28628533681632,
 'mdape': 0.21764848060697264,
 'rmse': 84390.73853846012,
 'r2': -1.6484418274393575,
 'raif_metric': 1.552534528099827}

In [ ]:
{'mape': 0.4006764913433361,
 'mdape': 0.28324961990050085,
 'rmse': 94343.39152646794,
 'r2': -4.286807955207906,
 'raif_metric': 2.6756861412533905}

{'mape': 0.2994072204832106,
 'mdape': 0.23136175216191418,
 'rmse': 85205.80509381538,
 'r2': -1.798582387807691,
 'raif_metric': 1.7313695017675064}

In [ ]:
{'mape': 0.45565440900238763,
 'mdape': 0.30249343647803606,
 'rmse': 100782.64525168539,
 'r2': -9.279126502012321,
 'raif_metric': 3.080882843995905}

In [72]:
X1.iloc[:,126:]

,realty_type,street,floor
0,110,S5845,-999
1,10,S27968,-999
2,10,S23597,-999
3,10,S10587,-999
4,10,S11220,-999
5,10,S6752,2.0
6,10,S6703,-999
7,10,S27593,-999
8,10,S19643,-999
9,10,S28820,-999


In [106]:
{'mape': 0.22629749264385002,
 'mdape': 0.19447444760686572,
 'rmse': 37925.78682309723,
 'r2': 0.6907862804815164,
 'raif_metric': 1.2450550091553048}
Xtest1 = Xtest1.fillna(-999)
test_data = Pool(data=Xtest1,
                  cat_features=raw_cat_features)
price0_preds = cat_model.predict(test_data)+1000

In [107]:
price0_preds

array([48461.5722999 , 42790.83136049, 38694.06343706, ...,
       60992.25486903, 50207.84320211, 55239.20223526])

In [108]:
pd.DataFrame({"id": [o.row['id'] for o in test_dataset.all_objects],
             "per_square_meter_price": price0_preds
             }).to_csv("no_overfit.csv", index=False)